In [19]:
! pip install tensorflow keras

  Using cached https://files.pythonhosted.org/packages/44/e1/dc0757b20b56c980b5553c1b5c4c32d378c7055ab7bfa92006801ad359ab/Keras-2.4.3-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/a1/63/8b9abaed8ce915dfccb856a25c8a47f96b508a1d7d60d2b7c21d5bb24207/scipy-1.5.3-cp37-cp37m-macosx_10_9_x86_64.whl
You are using pip version 19.0.3, however version 20.2.4 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [4]:
from ows_language_model.config import config

INTERIM_PATH = config.DATASET_DIR / 'interim'

with open(INTERIM_PATH / 'clean_docs.json', 'r') as f:
    english_docs = json.load(f)
    


In [35]:
shakespeare_text[0:10], flat_text[0:10]

('First Citi', ' Produced ')

In [54]:
flat_text = [" ".join(d) for d in english_docs[0:10]]

In [55]:
import tensorflow as tf
import numpy as np
import keras

In [85]:
vocab_size = 12000
tokenizer = keras.preprocessing.text.Tokenizer(num_words=vocab_size, char_level=False, oov_token='OOV_TOKEN')
tokenizer.fit_on_texts(flat_text)

In [86]:
max_id = min(len(tokenizer.word_index), vocab_size)
dataset_size = tokenizer.document_count

In [102]:
encoded = np.array(tokenizer.texts_to_sequences(flat_text))

In [103]:
train_size = len(encoded) * 90//100
n_steps = 50
window_length = n_steps + 1

In [104]:
single_doc = encoded[0]

In [124]:
batch_size=1
dataset = tf.data.Dataset.from_tensor_slices(single_doc)
print(dataset)
# create dataset of short, non-overlapping windows
# each window is a dataset in itself
dataset = dataset.window(window_length, shift=n_steps, drop_remainder=True)
# convert to flat dataset of {[window_1], [window_2]}
dataset = dataset.flat_map(lambda window : window.batch(window_length))
dataset = dataset.batch(batch_size)
print(dataset)
dataset = dataset.map(lambda windows : (windows[:, :-1], windows[:, 1:]))
print(dataset)
dataset = dataset.map(lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))
print(dataset)
dataset = dataset.prefetch(1)

<TensorSliceDataset shapes: (), types: tf.int32>
<BatchDataset shapes: (None, None), types: tf.int32>
<MapDataset shapes: ((None, None), (None, None)), types: (tf.int32, tf.int32)>
<MapDataset shapes: ((None, None, 12000), (None, None)), types: (tf.float32, tf.int32)>


In [125]:
model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, stateful=True,
                    dropout=0.2, recurrent_dropout=0.2,
                    batch_input_shape=[batch_size, None, max_id]),
    keras.layers.GRU(128, return_sequences=True, stateful=True,
                    dropout=0.2, recurrent_dropout=0.2,
                    batch_input_shape=[batch_size, None, max_id]),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id, activation="softmax"))
])

In [187]:
class ResetStatesCallback(keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs):
        out = complete_text(self.model, 'there once was', n_words=10, temperature = 1)
        print(out)
        out = complete_text(self.model, 'How many', n_words=10, temperature = 1)
        print(out)
        self.model.reset_states()

In [188]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
model.fit(dataset, epochs=10, callbacks=[ResetStatesCallback()])

there once was world my in of this any we still cheating in
How many that he lank some not “as he calling which analysis
Epoch 1/10
724/724 [==============================] - 942s 1s/step - loss: 6.3162
there once was basket as plane of do israel happiness shone business room
How many to rain to accepted of having steele room climax if
Epoch 2/10
724/724 [==============================] - 917s 1s/step - loss: 5.8629
there once was out you art which of day any write certainly was
How many them have perfectly came yet arms honour oil not north
Epoch 3/10
724/724 [==============================] - 906s 1s/step - loss: 5.6303
there once was that fir from of nine him forget one out “my
How many will one works orders until did all at that dozen
Epoch 4/10
724/724 [==============================] - 918s 1s/step - loss: 5.4049
there once was herself uses to of their knocks first out my neighbourhood
How many about of of for of herself die deliberate his every
Epoch 5/10
724/724 [==============

In [182]:
def next_word(model, text, temperature=1):
    X_new = tokenizer.texts_to_sequences([text])
    X_new = tf.one_hot(X_new, depth=max_id)
    y_proba = model.predict(X_new)[0, -1:, :]
    rescaled_logits = tf.math.log(y_proba)/temperature
    word_id = tf.random.categorical(rescaled_logits, num_samples=1)+1
    return tokenizer.sequences_to_texts(word_id.numpy())[0]

In [185]:
def complete_text(model, text, n_words=10, temperature = 1):
    for _ in range(n_words):
        text += ' '+ next_word(model, text, temperature=temperature)
    return text

This model seems to be working. There are a number of problems I have with it.
### Problems:
- The static graph aspect of this model means that it can only take in the same batch size as it was trained on, unless I do some weird cheap hack with stateless RNNs. Not very satisfying and very restrictive.
- The confusing data preparation steps actually make very little sense to me. The APIs don't let you really check what is going on, so I have to just take the Geron book's word for it.

### The plan now:

- Train a fastai model from the Howard and Gugger book